In [1]:
import pandas as pd
import numpy as np
import os
import warnings
from sklearn import preprocessing

from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

from sklearn.utils.class_weight import compute_class_weight
from sklearn import preprocessing

import torch
import torch.nn as nn
from torch.utils.data.sampler import SubsetRandomSampler
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data import Dataset

desired_frames = 1
desired_features = 2048

In [2]:
labels = pd.read_csv("/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/scene_labels.csv")

In [3]:
labels.head(5)

,SCENE,KEY,SPEAKER,SHOW,Sarcasm,Sarcasm_Type
0,1_10004,1_10004_u,SHELDON,BBT,0.0,NONE
1,1_10009,1_10009_u,PENNY,BBT,0.0,NONE
2,1_1001,1_1001_u,RAJ,BBT,0.0,NONE
3,1_1003,1_1003_u,HOWARD,BBT,1.0,PRO
4,1_10190,1_10190_u,SHELDON,BBT,0.0,NONE


In [4]:
scenes = list(pd.unique(labels["SCENE"]))

In [5]:
len(scenes)

1202

In [6]:
parent_dir = "/Users/yoshithaakunuri/Documents/CSCI535/Project/Final/data/features/utterances_final"
visual_features = {}
for i in range(len(scenes)):
    # Visual Features
    try:
        vf = np.load(os.path.join(parent_dir, "resnet_pool5_" + scenes[i] + ".npy"))
        #Global average pooling
        vf_p = np.apply_over_axes(np.mean, vf, [2, 3]) 
        vf_p = np.reshape(vf_p, (vf_p.shape[0],2048))
        if vf_p.shape[0] != 15:
            vf_p = np.pad(vf_p, [(0, 15 - vf_p.shape[0]), (0, 0)], mode='constant')
    except:
        vf_p = np.zeros((15, 2048))
    
    visual_features[scenes[i]] = vf_p

In [7]:
def get_model_data(video_features):
    model_data = pd.DataFrame(columns=['video_feature','sarcasm','sarcasm_type', 'speaker'])
    for index, row in labels.iterrows():
#         audio_key = row["SCENE"] + "_u.wav"
        model_data = model_data.append({'video_feature': video_features[row["SCENE"]],
                                    'sarcasm' : row["Sarcasm"],
                                    'sarcasm_type' : row["Sarcasm_Type"],
                                    'speaker' : row["SPEAKER"]},
                                  ignore_index=True)
    return model_data

In [8]:
def get_train_test_split(model_data, x_columns, y_column, stratify_column):
    X_train, X_test, Y_train, Y_test = train_test_split(
        model_data[x_columns],
        model_data[y_column],
        train_size=0.8, 
        test_size=0.2, 
        random_state=42, 
        shuffle=True,
        stratify=model_data[stratify_column])
    
    print("Train: ",X_train.shape, Y_train.shape,
      "Test: ",(X_test.shape, Y_test.shape))
    train_data = pd.merge(X_train, Y_train, left_index=True, right_index=True)
    test_data = pd.merge(X_test, Y_test, left_index=True, right_index=True)
    return train_data, test_data
   

In [9]:
class GRUTensorDataset(Dataset):
    def __init__(self, dataframe, speaker):
        self.data = dataframe
        self.speaker = speaker

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        if self.speaker:
            features = self.data.loc[index, 'padded_video_feature']
            a=np.empty((15,1))
            a.fill(self.data.loc[index, 'speaker_encode'])
            final_features = np.hstack((features, a))
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(final_features).float(), label
        else:
            features = self.data.loc[index, 'padded_video_feature']
            label = self.data.loc[index, 'sarcasm']
            return torch.from_numpy(features).float(), label

    def __getindexlist__(self):
        return list(self.data.index.values)
    
class GRUNetSD(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSD, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, 
                            self.hidden_dim).zero_()
        return hidden
    
class GRUNetSID(nn.Module):
    def __init__(self, input_dim, hidden_dim, 
                 output_dim, n_layers):
        super(GRUNetSID, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        
        self.gru = nn.GRU(input_dim, hidden_dim, 
                          n_layers, batch_first = True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, x, h):
        out, h = self.gru(x, h)
        out = self.softmax(self.fc(out[:,-1]))
        return out, h
    
    def init_hidden(self, batch_size):
#         weight = next(self.parameters()).data
#         hidden = weight.new(self.n_layers, batch_size, 
#                             self.hidden_dim).zero_()
        return torch.zeros(self.n_layers, batch_size, self.hidden_dim)
#         return hidden
    
def evaluateGRU(gru, review, size):
    hidden = gru.init_hidden(size)
    output, hidden = gru(review, hidden)
    return output

def categoryFromOutput(output):
    top_n, top_i = torch.max(output,dim=1)
    return top_i

def test_accuracy(gru, loader, size):
    actuals = []
    predictions = []
    for data, target in loader:
        output = evaluateGRU(gru, data, size)
        prediction_index = categoryFromOutput(output)
        predictions = prediction_index.tolist()
        actuals = target.tolist()
    return predictions, actuals
    
hidden_size = 15
output_size = 2
input_size_sid = 2048
input_size_sd = 2049
n_layers = 1

In [10]:
warnings.filterwarnings("ignore")

model_data = get_model_data(visual_features)
# Label Encode Speaker
le = preprocessing.LabelEncoder()
model_data['speaker_encode'] = le.fit_transform(model_data['speaker'])
model_data.head(5)

,video_feature,sarcasm,sarcasm_type,speaker,speaker_encode
0,"[[0.13835047, 0.2704592, 0.44648886, 0.1415337...",0.0,NONE,SHELDON,25
1,"[[0.46479157, 0.1813915, 0.22123067, 0.5245148...",0.0,NONE,PENNY,15
2,"[[0.253619, 0.25664786, 0.6646118, 0.4821793, ...",0.0,NONE,RAJ,21
3,"[[0.55624646, 0.16990338, 0.62457716, 0.209021...",1.0,PRO,HOWARD,7
4,"[[0.6140023, 0.4846397, 0.79425097, 0.13518682...",0.0,NONE,SHELDON,25


In [11]:
desired_length = 15

train_data, test_data = get_train_test_split(model_data, ['video_feature', 'speaker_encode'], 'sarcasm', 'sarcasm')
gru_train = train_data.copy()
gru_test = test_data.copy()
gru_train.reset_index(drop=True, inplace = True)
gru_test.reset_index(drop=True, inplace = True)

        
gru_train['padded_video_feature'] = gru_train.loc[:, 'video_feature']
# for index, row in gru_train.iterrows():
#     data_array = row['padded_video_feature']
#     new_array = []
#     for arr in data_array:
#         if arr.shape[0] < desired_length:
#             arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
#             new_array.append(arr)
#         else:
#             new_array.append(arr)
# #     gru_train.at[index, "padded_video_feature"] = np.transpose(np.array(new_array))
#     gru_train.at[index, "padded_video_feature"] = np.array(new_array)

gru_test['padded_video_feature'] = gru_test.loc[:, 'video_feature']
# for index, row in gru_test.iterrows():
#     data_array = row['padded_video_feature']
#     new_array = []
#     for arr in data_array:
#         if arr.shape[0] < desired_length:
#             arr = np.pad(arr, (0, desired_length - arr.shape[0]), 'constant')
#             new_array.append(arr)
#         else:
#             new_array.append(arr)
# #     gru_test.at[index, "padded_video_feature"] = np.transpose(np.array(new_array))
#     gru_test.at[index, "padded_video_feature"] = np.array(new_array)

gru_train["sarcasm"] = gru_train["sarcasm"].astype('int').to_numpy()
gru_test["sarcasm"] = gru_test["sarcasm"].astype('int').to_numpy()

Train:  (961, 2) (961,) Test:  ((241, 2), (241,))


### Speaker Independent

In [12]:
gru_train_tensor = GRUTensorDataset(gru_train[['padded_video_feature', 'sarcasm']], False)
gru_test_tensor = GRUTensorDataset(gru_test[['padded_video_feature', 'sarcasm']], False)

num_of_workers = 0
batch_size = 110
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices),
    drop_last = True
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices),
    drop_last = True
)

In [13]:
gru = GRUNetSD(input_size_sid, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSD(
  (gru): GRU(2048, 15, batch_first=True)
  (fc): Linear(in_features=15, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [14]:
n_epochs = 11
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(0)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "grumodel.pt")
#         test_min_loss = test_loss
    if(epoch%1 == 0):
        print("Epoch: " + str(epoch))
        test_loader = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader, gru_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)
        test_loader = torch.utils.data.DataLoader(gru_test_tensor, batch_size=batch_size, sampler=SubsetRandomSampler(test_indices))

Epoch: 0
              precision    recall  f1-score     support
0              0.000000  0.000000  0.000000  121.000000
1              0.497925  1.000000  0.664820  120.000000
accuracy       0.497925  0.497925  0.497925    0.497925
macro avg      0.248963  0.500000  0.332410  241.000000
weighted avg   0.247930  0.497925  0.331031  241.000000
Epoch: 1
              precision    recall  f1-score     support
0              0.000000  0.000000  0.000000  121.000000
1              0.497925  1.000000  0.664820  120.000000
accuracy       0.497925  0.497925  0.497925    0.497925
macro avg      0.248963  0.500000  0.332410  241.000000
weighted avg   0.247930  0.497925  0.331031  241.000000
Epoch: 2
              precision    recall  f1-score     support
0              0.000000  0.000000  0.000000  121.000000
1              0.497925  1.000000  0.664820  120.000000
accuracy       0.497925  0.497925  0.497925    0.497925
macro avg      0.248963  0.500000  0.332410  241.000000
weighted avg   0.2479

### Speaker Dependent

In [15]:
gru_train_tensor = GRUTensorDataset(gru_train[['padded_video_feature', 'speaker_encode','sarcasm']], True)
gru_test_tensor = GRUTensorDataset(gru_test[['padded_video_feature', 'speaker_encode','sarcasm']], True)

num_of_workers = 0
batch_size = 32
valid_size = 0.1

train_indices = list(range(len(gru_train_tensor)))
np.random.shuffle(train_indices)

test_indices = list(range(len(gru_test_tensor)))
np.random.shuffle(test_indices)

train_loader = torch.utils.data.DataLoader(
    gru_train_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(train_indices),
    drop_last=True
)

test_loader = torch.utils.data.DataLoader(
    gru_test_tensor, 
    batch_size=batch_size, 
    sampler=SubsetRandomSampler(test_indices),
    drop_last=True
)

In [16]:
gru = GRUNetSD(input_size_sd, hidden_size, output_size, n_layers)
print(gru)
criterion = nn.NLLLoss()
optimizer = torch.optim.Adam(gru.parameters(), lr=0.001)

GRUNetSD(
  (gru): GRU(2049, 15, batch_first=True)
  (fc): Linear(in_features=15, out_features=2, bias=True)
  (softmax): LogSoftmax(dim=1)
)


In [17]:
n_epochs = 21
    
test_min_loss = np.inf

for epoch in range(n_epochs):
    torch.manual_seed(42)
    train_loss = 0.0
    test_loss = 0.0
    gru.train()
    for data, target in train_loader:
        h = gru.init_hidden(batch_size)
        optimizer.zero_grad()
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()*data.size(0)

    gru.eval()
    for data, target in test_loader:
        if data.shape[1] < 44:
            continue
        h = gru.init_hidden(batch_size)
        output, h = gru(data, h.data)
        loss = criterion(output, target)
        test_loss += loss.item()*data.size(0)

    train_loss = train_loss / len(train_loader.dataset)
    test_loss = test_loss / len(test_loader.dataset)
    
#     if(epoch%20 == 0):
# #         print(f"Epoch: {epoch+1:02}")
# #         print("\tTraining Loss: {:.6f} \Test Loss: {:.6f}".format(train_loss, test_loss))
#     if test_loss <= test_min_loss:
# #         print("Test loss decreased ({:.6f} --> {:.6f}). Saving model...".format(test_min_loss, test_loss))
# #         torch.save(gru.state_dict(), "grumodel.pt")
#         test_min_loss = test_loss
    if(epoch%5 == 0):
        print("Epoch: " + str(epoch))
        test_loader = torch.utils.data.DataLoader(gru_test_tensor, batch_size=gru_test_tensor.__len__())
        predictions, actuals = test_accuracy(gru, test_loader, gru_test_tensor.__len__())
        print(pd.DataFrame(classification_report(actuals, predictions, output_dict=True)).T)
        test_loader = torch.utils.data.DataLoader(gru_test_tensor, batch_size=batch_size, sampler=SubsetRandomSampler(test_indices))

Epoch: 0
              precision    recall  f1-score     support
0              0.629213  0.462810  0.533333  121.000000
1              0.572368  0.725000  0.639706  120.000000
accuracy       0.593361  0.593361  0.593361    0.593361
macro avg      0.600791  0.593905  0.586520  241.000000
weighted avg   0.600909  0.593361  0.586299  241.000000
Epoch: 5
              precision    recall  f1-score    support
0              0.622449  0.504132  0.557078  121.00000
1              0.580420  0.691667  0.631179  120.00000
accuracy       0.597510  0.597510  0.597510    0.59751
macro avg      0.601434  0.597899  0.594128  241.00000
weighted avg   0.601521  0.597510  0.593974  241.00000
Epoch: 10
              precision    recall  f1-score     support
0              0.625000  0.537190  0.577778  121.000000
1              0.591241  0.675000  0.630350  120.000000
accuracy       0.605809  0.605809  0.605809    0.605809
macro avg      0.608120  0.606095  0.604064  241.000000
weighted avg   0.608190  0